#  Install required libraries

In [10]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
pip install google-cloud-storage

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 21.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   -------------------------------------- 123.5/123.5 kB 258.6 kB/s eta 0:00:00


In [ ]:
pip install db-dtypes
pip install google-cloud-bigquery-storage

In [1]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os


C:\Users\Himanshu\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Himanshu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Himanshu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\Himanshu\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Sample code for connection with bigquery and retreiving data

In [14]:
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/Himanshu/Desktop/Autogen/chatdb-project-54de4105bfb0_old.json'

In [15]:
from google.cloud import bigquery
bigquery_client = bigquery.Client()

In [16]:
QUERY = """
SELECT * FROM `chatdb-project.Dataset1.flights` LIMIT 10
  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df.head()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2006-08-26,DL,19790,LAX,CA,33.94,-118.4,LGA,NY,40.77,-73.87,1235,1325,50,2105,2138,33
1,2006-08-05,DL,19790,LAX,CA,33.94,-118.4,LGA,NY,40.77,-73.87,1235,1254,19,2105,2059,-6
2,2006-11-25,DL,19790,LAX,CA,33.94,-118.4,LGA,NY,40.77,-73.87,1235,1305,30,2105,2107,2
3,2006-07-22,DL,19790,LAX,CA,33.94,-118.4,LGA,NY,40.77,-73.87,1235,1418,103,2105,2253,108
4,2006-12-23,DL,19790,LAX,CA,33.94,-118.4,LGA,NY,40.77,-73.87,1235,1320,45,2105,2123,18


# ChatDB model with bigquery connection

In [3]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI

In [4]:
service_account_file = "C:/Users/Himanshu/Desktop/Autogen/chatdb-project-54de4105bfb0_old.json" # Change to where your service account key file is located
project = "chatdb-project"
dataset = "Dataset1"
table = "flights"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

print(sqlalchemy_url)

bigquery://chatdb-project/Dataset1?credentials_path=C:/Users/Himanshu/Desktop/Autogen/chatdb-project-54de4105bfb0_old.json


In [5]:
os.environ["OPENAI_API_KEY"] = "sk-*******"

In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
db = SQLDatabase.from_uri(sqlalchemy_url)
#llm = OpenAI(temperature= 0.8,openai_api_key="sk-*****",model_name= "gpt-3.5-turbo")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", max_tokens=1000)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
print(toolkit)

db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DBB70D0700> llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001DBB70AE310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001DBB70C3A60>, model_name='gpt-3.5-turbo-0613', temperature=0.0, openai_api_key='sk-******', openai_proxy='', max_tokens=1000)


C:\Users\Himanshu\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=True,
top_k=1000,
)

In [9]:
#agent_executor.run("how many columns are present in flights table? ")
agent_executor.run("how many airlines departed on 2006-10-14? ")


C:\Users\Himanshu\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQL Agent Executor chain...
I need to find the number of airlines that departed on a specific date. I should check the schema to see which table contains the information about departures. Then I can write a query to count the number of distinct airlines on that date.
Action: sql_db_schema
Action Input: departuresError: table_names {'departures'} not found in databaseI need to check the list of tables in the database to see if the table "departures" exists.
Action: sql_db_list_tables
Action Input: flightsThe table "departures" does not exist. I should check the schema of the "flights" table to see if it contains the information about departures.
Action: sql_db_schema
Action Input: flights
CREATE TABLE `flights` (
	`date` DATE, 
	`airline` STRING, 
	`airline_code` INT64, 
	`departure_airport` STRING, 
	`departure_state` STRING, 
	`departure_lat` FLOAT64, 
	`departure_lon` FLOAT64, 
	`arrival_airport` STRING, 
	`arrival_state` STRING, 
	`arrival_lat` FLOAT64, 
	`arrival_l

'15'